In [ ]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_zip(url, output_path):
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200: # 200은 서버측 응답코드, 정상적 응답을 서버로 부터 받았을때
        with open(output_path, 'wb') as f: # 이 경로로
            for chunk in response.iter_content(chunk_size=8192):# 8192건씩 반복적으로 읽음
                f.write(chunk)
        print(f"ZIP file downloaded to {output_path}")
    else:
        print(f"Failed to download. HTTP Response Code: {response.status_code}")

url = "http://www.manythings.org/anki/fra-eng.zip"
output_path = "fra-eng.zip"
download_zip(url, output_path)

path = os.getcwd()
zipfilename = os.path.join(path, output_path) # 현재 디렉토리 경로로 저장

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

ZIP file downloaded to fra-eng.zip


In [ ]:
pwd

'C:\\Users\\user\\Downloads\\jupyter notebook(machine learning)'

In [ ]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 227815


In [ ]:
lines

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !
...,...,...
227810,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
227811,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
227812,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
227813,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...


In [ ]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

,src,tar
31598,You're very timid.,Vous êtes fort craintifs.
24597,You have to help.,Vous devez aider.
32975,How do you like it?,Qu'en penses-tu ?
9807,We must hurry.,Il nous faut nous dépêcher.
29631,This will be easy.,Ça sera facile.
13163,Tom is a moron.,Tom est un idiot.
16317,Is anybody hurt?,Quiconque est-il blessé ?
54579,Can I ask you a favor?,Puis-je vous demander une faveur ?
47114,Find yourself a seat.,Trouve-toi un siège.
33656,I like candlelight.,J’aime la lumière des bougies.


In [ ]:
# src이 rnn으로 올라감
# teacher forcing에 대해 알아보기

![image.png](attachment:image.png)

In [ ]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
# \t : 시작, \n : 종료, 타켓(x) 앞뒤로 시작, 종료 설정
lines.sample(10)

,src,tar
43827,This belongs to you.,\t Ceci vous appartient. \n
23570,Tom sat silently.,\t Tom s'est assis en silence. \n
41387,I was already tired.,\t J'étais déjà fatigué. \n
19982,He needs a towel.,\t Il lui faut une serviette. \n
28257,Let me pay for it.,\t Laissez-moi le payer. \n
32073,Do you like apples?,\t Est-ce que tu aimes les pommes ? \n
45957,You're no different.,\t Vous n'êtes pas différent. \n
5952,Keep smiling.,\t Garde le sourire. \n
38965,Can I ask something?,\t Je peux poser une question ? \n
18721,You are naughty.,\t Vous êtes vilain. \n


In [ ]:
# 문자 집합 구축
src_vocab = set() # 구조는 set이라 중복이 안들어감
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

In [ ]:
src_vocab

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ï',
 '’',
 '€'}

In [ ]:
tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [ ]:
tar_vocab

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'Ô',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '‘',
 '’',
 '\u202f',
 '‽'}

In [ ]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 80
target 문장의 char 집합 : 104


In [ ]:
src_vocab=sorted(list(src_vocab)) # 정렬화, 리스트화, 아스키 코드 기준으로 정렬

In [ ]:
src_vocab # 공백등등 문자도 출력됨

[' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ï',
 '’',
 '€']

In [ ]:
tar_vocab = sorted(list(tar_vocab))

In [ ]:
tar_vocab

['\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'Ô',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '‘',
 '’',
 '\u202f',
 '‽']

In [ ]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)]) # enumerate로 각각으로 열거
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, 'ï': 77, '’': 78, '€': 79}


In [ ]:
print(tar_to_index)

{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 35, 'J': 36, 'K': 37, 'L': 38, 'M': 39, 'N': 40, 'O': 41, 'P': 42, 'Q': 43, 'R': 44, 'S': 45, 'T': 46, 'U': 47, 'V': 48, 'W': 49, 'X': 50, 'Y': 51, 'a': 52, 'b': 53, 'c': 54, 'd': 55, 'e': 56, 'f': 57, 'g': 58, 'h': 59, 'i': 60, 'j': 61, 'k': 62, 'l': 63, 'm': 64, 'n': 65, 'o': 66, 'p': 67, 'q': 68, 'r': 69, 's': 70, 't': 71, 'u': 72, 'v': 73, 'w': 74, 'x': 75, 'y': 76, 'z': 77, '\xa0': 78, '«': 79, '»': 80, 'À': 81, 'Ç': 82, 'É': 83, 'Ê': 84, 'Ô': 85, 'à': 86, 'â': 87, 'ç': 88, 'è': 89, 'é': 90, 'ê': 91, 'ë': 92, 'î': 93, 'ï': 94, 'ô': 95, 'ù': 96, 'û': 97, 'œ': 98, '\u2009': 99, '‘': 100, '’': 101, '\u202f': 102, '‽': 103}


# 영어 문장 인코딩

In [ ]:
encoder_input = []
# 모든 문자들에 대한 문장단위로 리스트로 이루어짐
# encoder_input [ 30,64],,[],,[]] 이런 식으로

In [ ]:
lines.src

0                           Go.
1                           Go.
2                           Go.
3                           Go.
4                           Hi.
                  ...          
59995    These are my students.
59996    These aren't my ideas.
59997    These books are Tom's.
59998    These books are heavy.
59999    These boxes are heavy.
Name: src, Length: 60000, dtype: object

In [ ]:
for line in lines.src:
    encoded_line=[]
    for c in line:       #G가 c에 먼저 읽어짐
        encoded_line.append(src_to_index[c])
    encoder_input.append(encoded_line)

In [ ]:
encoder_input

[[30, 64, 10],
 [30, 64, 10],
 [30, 64, 10],
 [30, 64, 10],
 [31, 58, 10],
 [31, 58, 10],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [46, 57, 64, 23],
 [46, 64, 72, 2],
 [46, 64, 72, 2],
 [46, 64, 72, 2],
 [27, 70, 52, 60, 2],
 [27, 70, 52, 60, 2],
 [27, 70, 52, 60, 2],
 [29, 58, 67, 54, 2],
 [31, 54, 61, 65, 2],
 [31, 58, 53, 54, 10],
 [31, 58, 53, 54, 10],
 [33, 70, 62, 65, 2],
 [33, 70, 62, 65, 10],
 [42, 69, 64, 65, 2],
 [42, 69, 64, 65, 2],
 [42, 69, 64, 65, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [25, 54, 56, 58, 63, 10],
 [25, 54, 56, 58, 63, 10],
 [30, 64, 1, 64, 63, 10],
 [30, 64, 1, 64, 63, 10],
 [30, 64, 

In [ ]:
decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 48, 52, 3, 4, 3, 2], [1, 3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [1, 3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [1, 3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [1, 3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [ ]:
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0: # \t(역슬래시 제거)
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 48, 52, 3, 4, 3, 2], [3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [ ]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 76


In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [ ]:
# 문자단위로 원핫인코딩하기(categorical)
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [ ]:
encoder_input.shape

(60000, 22, 80)

In [ ]:
decoder_input.shape

(60000, 76, 104)

In [ ]:
decoder_target.shape # 정답

(60000, 76, 104)

In [ ]:
# 이제 시퀀스 투 시퀀스로 훈련해야함

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
# Seq2Seq 인코더 설계

In [ ]:
encoder_inputs = Input(shape=(None, src_vocab_size))

In [ ]:
src_vocab_size  # 입력되는 문장의 문자의 종류 = 차원 = 문자벡터의 크기
               # 문자의 길이는 일정하지 않으므로 위에서 22로 맞춰줌
              # 각각의 문장에 등장하는 문자의 종류는 80가지
             # 문자하나당 80차원

80

![image-2.png](attachment:image-2.png)
- return_state 알아보기

#### return_state=True
![image.png](attachment:image.png)

In [ ]:
encoder_lstm=LSTM(units=256, return_state=True)
# LSTM에서 출력이 차원, False가 디폴트값, 256차원으로 출력

In [ ]:
encoder_outputs, state_h, state_c=encoder_lstm(encoder_inputs)# encoder_lstm에 집어 넣으라는 뜻
#state_h(은닉 상태), state_c(셀 상태)

In [ ]:
encoder_states = [state_h, state_c] # encoder_states는 컨텍스트 벡터가 됨
# encoder_states가 디코더쪽으로 이동

In [ ]:
# Seq2Seq 디코더 설계

In [ ]:
decoder_inputs = Input(shape=(None, tar_vocab_size))

In [ ]:
decoder_lstm = LSTM(units=256, return_state=True, return_sequences=True)
# 번역 결과때문에 return_sequences가 true가 되야함
# initial_state=encoder_states : decoder_lstm의 초기 상태값
#디코더 inputs이 lstm으로 올라가야하며 target, encoder_states가 올라가야됨
# encoder_states를 전달받아야됨

![image.png](attachment:image.png)

In [ ]:
decoder_outputs, _, _ =decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더는 은닉 상태, 셀 상태 정보가 필요 없음(훈련과정에서는), 256차원

![image-2.png](attachment:image-2.png)
- 프랑스어에 속하는 문자 집합의 크기
- 소프트맥스 함수 이용

In [ ]:
import numpy as np

In [ ]:
#decoder_outputs와 decoder_softmax_layer를 fully-connected해야 학습됨

In [ ]:
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')

In [ ]:
decoder_outputs=decoder_softmax_layer(decoder_outputs)

In [ ]:
# 모델 설계
model = Model([encoder_inputs, decoder_inputs],decoder_outputs) # encoder_input, decoder_input를 각각 입려해야됨 둘다 입력으로 사용됨

In [ ]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [ ]:
model.fit(x=[encoder_input,decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
750/750 [==============================] - 119s 156ms/step - loss: 0.8444 - val_loss: 0.7688
Epoch 2/40
750/750 [==============================] - 121s 162ms/step - loss: 0.5714 - val_loss: 0.6578
Epoch 3/40
750/750 [==============================] - 119s 159ms/step - loss: 0.5040 - val_loss: 0.5983
Epoch 4/40
750/750 [==============================] - 122s 163ms/step - loss: 0.4598 - val_loss: 0.5577
Epoch 5/40
750/750 [==============================] - 126s 168ms/step - loss: 0.4254 - val_loss: 0.5213
Epoch 6/40
750/750 [==============================] - 128s 171ms/step - loss: 0.3956 - val_loss: 0.4902
Epoch 7/40
750/750 [==============================] - 117s 156ms/step - loss: 0.3740 - val_loss: 0.4737
Epoch 8/40
750/750 [==============================] - 124s 165ms/step - loss: 0.3572 - val_loss: 0.4574
Epoch 9/40
750/750 [==============================] - 122s 163ms/step - loss: 0.3429 - val_loss: 0.4377
Epoch 10/40
750/750 [==============================] - 121s 162m

In [ ]:
# 테스트 과정
1. 번역 대상 입력문장이 인코더에 들어감 -> 은닉/셀 상태 -> 디코더
2. 디코더의 입력 시그널이 전달(sos, \t)
3. 디코더는 다음 문자 예측 가능

![image.png](attachment:image.png)

In [ ]:
# 테스트 과정에서의 디코더 정의
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [ ]:
# 테스트 과정에서의 인코더 정의
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

      # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
            len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스, 문장들의 번호
    # 예를 들어 hi가 3번 이런식으로 이해하면 됨
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

1/1 [==============================] - 0s 28ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Pars ! 
1/1 [==============================] - 0s 17ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: Salut ! 
1/1 [==============================] - 0s 21ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: J'ai pigé ! 
번역 문장: Dégage ! 
1/1 [==============================] - 0s 25ms/step
-----------------------------------
입력 문장: Go home.
정답 문장: Rentre à la maison. 
번역 문장: Allez ! 
1/1 [==============================] - 0s 26ms/step
-----------------------------------
입력 문장: Get going.
정답 문장: En avant. 
번역 문장: Déparez-vous ! 


In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq,verbose=0)
  # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ""
  # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value,verbose=0)
    # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
    # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char
    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
            len(decoded_sentence) > max_tar_len):
            stop_condition = True
        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.
        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]
    return decoded_sentence

In [ ]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스, 문장들의 번호
    # 예를 들어 hi가 3번 이런식으로 이해하면 됨
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Pars ! 
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: Salut ! 
-----------------------------------
입력 문장: Got it!
정답 문장: J'ai pigé ! 
번역 문장: Dégage ! 
-----------------------------------
입력 문장: Go home.
정답 문장: Rentre à la maison. 
번역 문장: Allez ! 
-----------------------------------
입력 문장: Get going.
정답 문장: En avant. 
번역 문장: Déparez-vous ! 
